# Regression

In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

market_data = pd.read_csv("../processed/market_data.csv", parse_dates=["Date"])
market_data = market_data.set_index("Date").sort_index()

import statsmodels.api as sm

X = market_data[
    ["sp500_logret", "usdindex_logret", "inflation", "m2_growth", "fedfunds"]
]
X = sm.add_constant(X)

y = market_data["btc_logret"]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             btc_logret   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     6.284
Date:                Wed, 28 Jan 2026   Prob (F-statistic):           3.19e-05
Time:                        00:45:37   Log-Likelihood:                 58.824
No. Observations:                 128   AIC:                            -105.6
Df Residuals:                     122   BIC:                            -88.53
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0730      0.034      2.176      0.031       0.007       0.139
sp500_logret        2.2707      0.436      5.210      0.000       1.408       3.134
usdindex_logret     1.1569      1.264      0.915      0.362      -1.346       3.660
inflation         -10.2818      5.688     -1.808      0.073     -21.542       0.979
m2_growth          -1.0128      2.032     -0.498      0.619      -5.035       3.009
fedfunds           -0.0083      0.008     -1.020      0.310      -0.024       0.008
==============================================================================
Omnibus:                       15.287   Durbin-Watson:                   1.330
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.991
Skew:                           0.735   Prob(JB):                     0.000124
Kurtosis:                       4.102   Cond. No.                     1.18e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
import pandas as pd
from pathlib import Path

def signif_stars(p):
    if p < 0.01:
        return "***"
    elif p < 0.05:
        return "**"
    elif p < 0.10:
        return "*"
    else:
        return ""

reg_table = pd.DataFrame({
    "Variable": model.params.index,
    "Koeffizient": model.params.values,
    "Std.-Fehler": model.bse.values,
    "p-Wert": model.pvalues.values,
})

reg_table["Signifikanz"] = reg_table["p-Wert"].apply(signif_stars)

reg_table = reg_table.round(4)

# Export-Pfad
TABLE_DIR = Path("../tables")
TABLE_DIR.mkdir(parents=True, exist_ok=True)

# LaTeX-Export
reg_table.to_latex(
    TABLE_DIR / "regression_results.tex",
    float_format="%.4f",
    caption=None,
    label=None,
    escape=True,
    index=False
)


# ETH Robust

In [15]:
import statsmodels.api as sm


market_data_eth = pd.read_csv("../processed/market_data_eth.csv", parse_dates=["Date"])
market_data_eth = market_data_eth.set_index("Date").sort_index()

X = market_data_eth[
    ["sp500_logret", "usdindex_logret", "inflation", "m2_growth", "fedfunds"]
]
X = sm.add_constant(X)

y = market_data_eth["eth_logret"]
model_eth = sm.OLS(y, X).fit()
model_eth.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             eth_logret   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     8.295
Date:                Wed, 28 Jan 2026   Prob (F-statistic):           1.86e-06
Time:                        01:17:39   Log-Likelihood:                 25.210
No. Observations:                  94   AIC:                            -38.42
Df Residuals:                      88   BIC:                            -23.16
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0380      0.062      0.610      0.543      -0.086       0.162
sp500_logret        3.1752      0.576      5.511      0.000       2.030       4.320
usdindex_logret    -0.2787      1.857     -0.150      0.881      -3.970       3.412
inflation          -5.0788      8.452     -0.601      0.549     -21.876      11.719
m2_growth           0.5390      2.835      0.190      0.850      -5.096       6.174
fedfunds           -0.0125      0.013     -0.968      0.336      -0.038       0.013
==============================================================================
Omnibus:                        1.433   Durbin-Watson:                   1.362
Prob(Omnibus):                  0.488   Jarque-Bera (JB):                0.907
Skew:                           0.195   Prob(JB):                        0.635
Kurtosis:                       3.283   Cond. No.                     1.43e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
import pandas as pd
from pathlib import Path

def signif_stars(p):
    if p < 0.01:
        return "***"
    elif p < 0.05:
        return "**"
    elif p < 0.10:
        return "*"
    else:
        return ""

reg_table = pd.DataFrame({
    "Variable": model_eth.params.index,
    "Koeffizient": model_eth.params.values,
    "Std.-Fehler": model_eth.bse.values,
    "p-Wert": model_eth.pvalues.values,
})

reg_table["Signifikanz"] = reg_table["p-Wert"].apply(signif_stars)

reg_table = reg_table.round(4)

# Export-Pfad
TABLE_DIR = Path("../tables")
TABLE_DIR.mkdir(parents=True, exist_ok=True)

# LaTeX-Export
reg_table.to_latex(
    TABLE_DIR / "regression_results_eth.tex",
    float_format="%.4f",
    caption=None,
    label=None,
    escape=True,
    index=False
)
